In [1]:
class AffineCipher:
    def __init__(self):
        self.mod = 26
        self.lower_char_ord = ord('a')
        self.upper_char_ord = ord('A')
        
    def encrypt(self, plaintext: str, a: int, b: int):
        """
        E(x) = (a*x + b) mod m
        """
        ciphertext = ""
        
        if not self.modinv(a, self.mod):
            raise Exception("modular inverse doesn't exist")

        for char in plaintext:
            if not self._is_special_char(char):
                if char.isupper():
                    ciphertext += chr(
                        ((ord(char) - self.upper_char_ord) * a + b) % self.mod + self.upper_char_ord
                    )
                else:
                    ciphertext += chr(
                        ((ord(char) - self.lower_char_ord) * a + b) % self.mod + self.lower_char_ord
                    )
            else:
                ciphertext += char

        return ciphertext

    def decrypt(self, ciphertext: str, a: int, b: int):
        """
        D(x)=a^-1 * (x - b) mod m
        """
        plaintext = ""
        a_inversed = self.modinv(a, self.mod)
        
        for char in ciphertext:
            if not self._is_special_char(char):
                if char.isupper():
                    plaintext += chr(
                        (a_inversed * (ord(char) - self.upper_char_ord - b)) % self.mod + self.upper_char_ord
                    )
                else:
                    plaintext += chr(
                        (a_inversed * (ord(char) - self.lower_char_ord - b)) % self.mod + self.lower_char_ord
                    )
            else:
                plaintext += char

        return plaintext

    def egcd(self, n: int, m: int):
        """
        Extended Euclidean Algorithm 
        """
        if m == 0:
            return n, 0, 1

        gcd, x, y = self.egcd(m, n % m)

        x1 = y - (n//m) * x
        y1 = x
        return gcd, x1, y1
    
    def modinv(self, a: int, m: int): 
        """
        D(x)=a^-1 * (x - b) mod m
        a: key
        m: mod %
        """
        gcd, x, y = self.egcd(a, m) 
        if gcd != 1:
            return None
        else: 
            return y % m
    
    def analyze_frequencies(self, text: str):
        char_frequency = {}
        for char in text:
            if not self._is_special_char(char):
                if char.lower() in char_frequency.keys():
                    char_frequency[char.lower()] += 1
                else:
                    char_frequency[char.lower()] = 1

        return char_frequency
    
    def hack_by_frequencies(self, ciphertext: str, train_text: str):
        encrypted_freq = self.analyze_frequencies(ciphertext)
        train_freq = self.analyze_frequencies(train_text)
        encrypted_freq_sorted = sorted(encrypted_freq, key=lambda x: encrypted_freq[x], reverse=True)
        train_freq_sorted = sorted(train_freq, key=lambda x: train_freq[x], reverse=True)
        decryption_map = dict(zip(encrypted_freq_sorted, train_freq_sorted))
        plaintext = ""
        for char in ciphertext:
            if not self._is_special_char(char):                
                if char.isupper():
                    plaintext += decryption_map[char.lower()].upper()
                else:
                    plaintext += decryption_map[char]
            else:
                plaintext += char

        return plaintext
    
    
    def _is_special_char(self, char: chr):
        return not (
            self.upper_char_ord <= ord(char) <= self.upper_char_ord + 25 or
            self.lower_char_ord <= ord(char) <= self.lower_char_ord + 25
        )

In [2]:
text = """By this time she had found her way into a tidy little room with
a table in the window, and on it (as she had hoped) a fan and two
or three pairs of tiny white kid gloves:  she took up the fan and
a pair of the gloves, and was just going to leave the room, when
her eye fell upon a little bottle that stood near the looking-
glass.  There was no label this time with the words `DRINK ME,'
but nevertheless she uncorked it and put it to her lips.  `I know
SOMETHING interesting is sure to happen,' she said to herself,
`whenever I eat or drink anything; so I'll just see what this
bottle does.  I do hope it'll make me grow large again, for
really I'm quite tired of being such a tiny little thing!'

  It did so indeed, and much sooner than she had expected:
before she had drunk half the bottle, she found her head pressing
against the ceiling, and had to stoop to save her neck from being
broken.  She hastily put down the bottle, saying to herself
`That's quite enough--I hope I shan't grow any more--As it is, I
can't get out at the door--I do wish I hadn't drunk quite so
much!'

  Alas! it was too late to wish that!  She went on growing, and
growing, and very soon had to kneel down on the floor:  in
another minute there was not even room for this, and she tried
the effect of lying down with one elbow against the door, and the
other arm curled round her head.  Still she went on growing, and,
as a last resource, she put one arm out of the window, and one
foot up the chimney, and said to herself `Now I can do no more,
whatever happens.  What WILL become of me?'
"""
book = open("alice_in_wonderland.txt", "r").read()

### Encrypt

In [3]:
affine = AffineCipher()
a = 5
b = 8

In [4]:
ciphertext = affine.encrypt(text, a, b)
print(ciphertext)

Ny zrwu zwqc urc rix haevx rcp oiy wvza i zwxy lwzzlc paaq owzr
i zinlc wv zrc owvxao, ivx av wz (iu urc rix rafcx) i hiv ivx zoa
ap zrpcc fiwpu ah zwvy orwzc gwx mlajcu:  urc zaag ef zrc hiv ivx
i fiwp ah zrc mlajcu, ivx oiu beuz mawvm za lcijc zrc paaq, orcv
rcp cyc hcll efav i lwzzlc nazzlc zriz uzaax vcip zrc laagwvm-
mliuu.  Zrcpc oiu va lincl zrwu zwqc owzr zrc oapxu `XPWVG QC,'
nez vcjcpzrclcuu urc evsapgcx wz ivx fez wz za rcp lwfu.  `W gvao
UAQCZRWVM wvzcpcuzwvm wu uepc za riffcv,' urc uiwx za rcpuclh,
`orcvcjcp W ciz ap xpwvg ivyzrwvm; ua W'll beuz ucc oriz zrwu
nazzlc xacu.  W xa rafc wz'll qigc qc mpao lipmc imiwv, hap
pcilly W'q kewzc zwpcx ah ncwvm uesr i zwvy lwzzlc zrwvm!'

  Wz xwx ua wvxccx, ivx qesr uaavcp zriv urc rix ctfcszcx:
nchapc urc rix xpevg rilh zrc nazzlc, urc haevx rcp rcix fpcuuwvm
imiwvuz zrc scwlwvm, ivx rix za uzaaf za uijc rcp vcsg hpaq ncwvm
npagcv.  Urc riuzwly fez xaov zrc nazzlc, uiywvm za rcpuclh
`Zriz'u kewzc cvaemr--W rafc W uriv'z mpao ivy qap

### Decrypt

#### Decrypt by known key

In [5]:
encrypted_text = affine.decrypt(ciphertext, a, b)
print(encrypted_text)

By this time she had found her way into a tidy little room with
a table in the window, and on it (as she had hoped) a fan and two
or three pairs of tiny white kid gloves:  she took up the fan and
a pair of the gloves, and was just going to leave the room, when
her eye fell upon a little bottle that stood near the looking-
glass.  There was no label this time with the words `DRINK ME,'
but nevertheless she uncorked it and put it to her lips.  `I know
SOMETHING interesting is sure to happen,' she said to herself,
`whenever I eat or drink anything; so I'll just see what this
bottle does.  I do hope it'll make me grow large again, for
really I'm quite tired of being such a tiny little thing!'

  It did so indeed, and much sooner than she had expected:
before she had drunk half the bottle, she found her head pressing
against the ceiling, and had to stoop to save her neck from being
broken.  She hastily put down the bottle, saying to herself
`That's quite enough--I hope I shan't grow any mor

#### Decrypt by the order of most frequent letters

In [6]:
hacked_text = affine.hack_by_frequencies(ciphertext=ciphertext, train_text=book)
print(hacked_text)

Mb this tice she hnl fogal her unb iato n tilb dittde rooc uith
n tnmde ia the uialou, nal oa it (ns she hnl hoyel) n fna nal tuo
or three ynirs of tiab uhite kil wdoves:  she took gy the fna nal
n ynir of the wdoves, nal uns jgst woiaw to denve the rooc, uhea
her ebe fedd gyoa n dittde mottde thnt stool aenr the dookiaw-
wdnss.  There uns ao dnmed this tice uith the uorls `LRIAK CE,'
mgt aeverthedess she gaporkel it nal ygt it to her diys.  `I kaou
SOCETHIAW iaterestiaw is sgre to hnyyea,' she snil to hersedf,
`uheaever I ent or lriak nabthiaw; so I'dd jgst see uhnt this
mottde loes.  I lo hoye it'dd cnke ce wrou dnrwe nwnia, for
renddb I'c qgite tirel of meiaw sgph n tiab dittde thiaw!'

  It lil so ialeel, nal cgph sooaer thna she hnl exyeptel:
mefore she hnl lrgak hndf the mottde, she fogal her henl yressiaw
nwniast the peidiaw, nal hnl to stooy to snve her aepk froc meiaw
mrokea.  She hnstidb ygt loua the mottde, snbiaw to hersedf
`Thnt's qgite eaogwh--I hoye I shna't wrou nab cor